# A try of unsupervised learning

Here we use Baum–Welch algorithm to estimate the parameters of our HMM, it is a type of EM algorithm, we set the parameters with random initial conditions at first, then we compute alpha(Forward procedure) and beta(Backward procedure), like what we do in supervised learning. To update the parametres, we have to compute 2 new temporary variables(gama and epsilon), then update according to these 2 variables.

In [1]:
import pickle
train10 = pickle.load( open( "./typos-data/train10.pkl", "rb" ) )
train20 = pickle.load( open( "./typos-data/train20.pkl", "rb" ) )
test10 = pickle.load( open( "./typos-data/test10.pkl", "rb" ) )
test20 = pickle.load( open( "./typos-data/test20.pkl", "rb" ) )
print("length of train10 : ",len(train10))
print("length of train20 : ",len(train20))
print("length of test10 : ",len(test10))
print("length of test20 : ",len(test20))

length of train10 :  29057
length of train20 :  27184
length of test10 :  1501
length of test20 :  3374


In [2]:
observation_list=[]
state_list=[]
for mot in train10:
    for (l1,l2) in mot:
        observation_list.append(l1)
        state_list.append(l2)
observation_list=list(set(observation_list))
state_list=list(set(state_list))

In [78]:
import nltk
from numpy import array, ones, zeros, multiply
import numpy as np
import sys
from pandas import Series, DataFrame

# construc the first order HMM Model

UNK = "<unk>"
UNKid = 0
epsilon = 1e-100

class HMM:
        def __init__(self, state_list, observation_list, train = None,test = None,
                 transition_proba = None,
                 observation_proba = None,
                 initial_state_proba = None, smoothing_obs = 0.01):
            """
            Builds a Hidden Markov Model
            * state_list is the list of state symbols [q_0...q_(N-1)]
            * observation_list is the list of observation symbols [v_0...v_(M-1)]
            * transition_proba is the transition probability matrix
                [a_ij] a_ij = Pr(Y_(t+1)=q_i|Y_t=q_j)
            * observation_proba is the observation probablility matrix
                [b_ki] b_ki = Pr(X_t=v_k|Y_t=q_i)
            * initial_state_proba is the initial state distribution
                [pi_i] pi_i = Pr(Y_0=q_i)"""
            print("HMM creating with: ")
            self.N = len(state_list)       # number of states
            self.M = len(observation_list) # number of possible emissions
            print(str(self.N)+" states")
            print(str(self.M)+" observations")
            self.train=train
            self.omega_Y = state_list
            self.omega_X = observation_list
            if transition_proba is None:
                self.transition_proba = zeros( (self.N, self.N), float) 
            else:
                self.transition_proba=transition_proba
            if observation_proba is None:
                self.observation_proba = zeros( (self.N, self.M), float) 
            else:
                self.observation_proba=observation_proba
            if initial_state_proba is None:
                self.initial_state_proba = zeros( (self.N,), float ) 
            else:
                self.initial_state_proba=initial_state_proba
            self.make_indexes() # build indexes, i.e the mapping between token and int
            self.data2index()
#             self.calculer_pi()
#             self.calculer_A()
#             self.calculer_B()
            self.Baum_Welch()
        def make_indexes(self):
            """Creates the reverse table that maps states/observations names
            to their index in the probabilities array"""
            self.Y_index = {}
            for i in range(self.N):
                self.Y_index[self.omega_Y[i]] = i
            self.X_index = {}
            for i in range(self.M):
                self.X_index[self.omega_X[i]] = i
        
        def data2index(self):
            self.Y_char={}
            for i in self.Y_index:
                self.Y_char[self.Y_index[i]]=i
        
#         def calculer_pi(self):
#             for word in self.train:
#                 self.initial_state_proba[self.Y_index[word[0][1]]]+=1
# #             print(self.initial_state_proba)
#             self.initial_state_proba/=len(self.train)
    
#         def calculer_A(self):
#             s=0
#             for word in self.train:
#                 for i in range(len(word)-1):
#                     s+=1
#                     w1=self.Y_index[word[i][1]]
#                     w2=self.Y_index[word[i+1][1]]
#                     self.transition_proba[w1][w2]+=1
#             tmp=self.transition_proba.T
#             self.transition_proba=(tmp/self.transition_proba.sum(axis=1)).T
# #             self.transition_proba/=self.transition_proba.sum(axis=0)
        
#         def calculer_B(self):
#             s=0
#             for word in self.train:
#                 for i in range(len(word)):
#                     s+=1
#                     w1=self.Y_index[word[i][1]]
#                     w2=self.X_index[word[i][0]]
#                     self.observation_proba[w1][w2]+=1
#             tmp=self.observation_proba.T
#             self.observation_proba=(tmp/self.observation_proba.sum(axis=1)).T
        def initialize_parametres(self):
            self.transition_proba = np.random.rand(26, 26)
            self.observation_proba = np.random.rand(26, 26)
            self.initial_state_proba = np.random.rand(26,)
            self.transition_proba/=self.transition_proba.sum(axis=1)
            self.observation_proba/=self.observation_proba.sum(axis=1)
            self.initial_state_proba/=self.initial_state_proba.sum()

            
        def calculer_alpha(self,word):            
            alpha=np.zeros((len(self.Y_index),len(word)),float)
            alpha[:,0]=self.initial_state_proba*self.observation_proba[:,self.X_index[word[0][0]]]
            if len(word)==1:
                return alpha
            for i in range(1,len(word)):
                alpha[:,i]=(alpha[:,i-1]*self.transition_proba.T).sum(axis=1)*\
                self.observation_proba[:,self.X_index[word[i][0]]]
            return alpha
        
        def calculer_beta(self,word):
            beta=np.zeros((len(self.Y_index),len(word)),float)
            beta[:,-1]=1
            if len(word)==1:
                return beta
            for i in range(len(word)-2,-1,-1):
                beta[:,i]=(self.transition_proba*self.observation_proba[:,self.X_index[word[i+1][0]]]*\
                           beta[:,i+1]).sum(axis=1)
            return beta
        def calculer_epsilon(self,alpha,beta,word):
            epsilon = np.zeros([26,26,len(word)])
            for t in range(len(word)-1):
                for i in range(26):
                    for j in range(26):
                        epsilon[i,j,t]=alpha[i,t]*self.transition_proba[i][j]*\
                        self.observation_proba[j,self.X_index[word[t+1][0]]]*beta[j,t+1]\
                
                            
            epsilon/=sum(sum(epsilon))
            return epsilon
        def para_update(self,word,gamma,epsilon):

            self.initial_state_proba = gamma[:,0]
            for i in range(26):
                for j in range(26):
                    self.transition_proba[i,j] = sum(epsilon[i,j,:]) / sum(gamma[i,:])

            for j in range(26):
                for k in range(26):
                    self.observation_proba[j,k] = sum([gamma[j][t] for t in range(len(word)-1)\
                                                       if k == self.X_index[word[t][0]]])/sum(gamma[j,:])



                        
        def Baum_Welch(self):
            self.initialize_parametres()
            epochs=30
            for i in range(epochs):
                for word in self.train:
                    if len(word)<2:
                        continue
                    alpha = self.calculer_alpha(word)
                    beta = self.calculer_beta(word)
                    gama = alpha * beta
#                     gama/= gama.sum(axis=1)
                    gama/=sum(gama)
                    epsilon = self.calculer_epsilon(alpha,beta,word)
                    gama=gama[:,:-1]
                    self.para_update(word,gama,epsilon)

                    
        
        def FB(self,alpha,beta):
            prob = alpha * beta
            index=list(prob.argmax(axis=0))
#             print(index)
            r=[self.Y_char[i] for i in index]
            return r
    
        def viterbi(self,word):
            N=len(self.Y_index)
            delta=np.zeros(N, float)
            delta_t=np.zeros(N, float)
            tmp=np.zeros(N, float)
            index=np.zeros((len(word),N), int)
            delta=self.initial_state_proba*self.observation_proba[:,self.X_index[word[0][0]]]
            for t in range(1, len(word)):
                p=self.X_index[word[t][0]]
                for j in range(N):
                    tmp=delta*self.transition_proba[:,j]
                    index[t,j]=tmp.argmax()
                    delta_t[j]=tmp[index[t,j]]*self.observation_proba[j,p]
                delta, delta_t = delta_t, delta
            result=[delta.argmax()]
            for i in index[-1:0:-1]:
                result.append(i[result[-1]])
            result.reverse()
            r=[self.Y_char[i] for i in result]
            return r
        def score_FB(self, test):
            s=0
            e=0
            correct=0
            creat=0
            for word in test:
                alpha=self.calculer_alpha(word)
                beta=self.calculer_beta(word)
                nword=self.FB(alpha,beta)
                for i,j in zip(word,nword):
                    s+=1
                    if i[1]!=j:
                        e+=1
                    if i[0]!=i[1] and i[1]==j:
                        correct+=1
                    if i[1]!=j and i[0]==i[1]:
                        creat+=1
            return {"Error rate":e/s, "Errors correct":correct, "Errors creat":creat}
        
        def score_viterbi(self, test):
            s=0
            e=0
            correct=0
            creat=0
            for word in test:
                nword=self.viterbi(word)
                for i,j in zip(word,nword):
                    s+=1
                    if i[1]!=j:
                        e+=1
                    if i[0]!=i[1] and i[1]==j:
                        correct+=1
                    if i[1]!=j and i[0]==i[1]:
                        creat+=1
            return {"Error rate":e/s, "Errors correct":correct, "Errors creat":creat}
        def baseline(self,test):
            s=0
            e=0
            for word in test:
                for l in word:
                    s+=1
                    if l[0]!=l[1]:
                        e+=1
            return e/s

In [81]:
model1=HMM(state_list,observation_list,train10,None)
print("Test10, trained with train10:")
print("Baseline:",model1.baseline(test10))
print("FB for test10:",model1.score_FB(test10))
print("Viterbe for test10:",model1.score_viterbi(test10))

HMM creating with: 
26 states
26 observations


/home/gjx/.local/lib/python3.6/site-packages/ipykernel_launcher.py:134: RuntimeWarning: invalid value encountered in true_divide


Test10, trained with train10:
Baseline: 0.10177595628415301
FB for test10: {'Error rate': 0.9780054644808743, 'Errors correct': 13, 'Errors creat': 6427}
Viterbe for test10: {'Error rate': 0.9780054644808743, 'Errors correct': 13, 'Errors creat': 6427}


The process of computation is very long, and the results are very poor. Maybe the number of trainings is not enough, or it is more likely that somewhere of our program is wrong. Anyway, through this experiment, our understanding of unsupervised learning and EM algorithm is much deeper.